In [1]:
# Importing libraries
import lancedb
import pyarrow as pa
from transformers import CLIPModel, CLIPProcessor
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import open_clip
from PIL import Image

c:\Users\minhh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
# Load CLIP model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14-378-quickgelu', pretrained='dfn5b')
model.to(device)

c:\Users\minhh\AppData\Local\Programs\Python\Python311\Lib\site-packages\open_clip\factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(chec

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-31): 32 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=Tru

In [3]:
# Dataset and dataloader
class KeyframesDataset(Dataset):
    def __init__(self):
        self.PATH = "E:\\Downloads\\atm24949_withsound\\keyframes"
        print(f"Image path for dataset: {self.PATH}\\*\\*")
        self.images_paths = glob.glob(f"{self.PATH}\\*\\*")
        
    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.images_paths[idx]))
        video_name = self.images_paths[idx].rsplit('\\', 2)[1]
        image_name = self.images_paths[idx].rsplit('\\', 1)[1]
        frame_idx = self.images_paths[idx].rsplit('\\', 1)[1][:-4].rsplit('-', 1)[1]
        return image, video_name, image_name, frame_idx, self.images_paths[idx].split('\\', 3)[3]

dataset = KeyframesDataset()
dataloader = DataLoader(dataset, batch_size=128, shuffle=False)
print("Number of images: ", len(dataset))
print("Number of batches: ", len(dataloader))

Image path for dataset: E:\Downloads\atm24949_withsound\keyframes\*\*
Number of images:  368484
Number of batches:  2879


In [4]:
# Create lancedb instance
lancedb_instance = lancedb.connect("database.lance")
TABLE_NAME = "patch14v2_extended_openclip"
if TABLE_NAME in lancedb_instance.table_names():
    database = lancedb_instance[TABLE_NAME]
    print(f"Warning: Table {TABLE_NAME} already exists!")
else:
    schema = pa.schema([
        pa.field("embedding", pa.list_(pa.float32(), 1024)),
        pa.field("video_name", pa.string()),
        pa.field("image_name", pa.string()),
        pa.field("frame_idx", pa.int32()),
        pa.field("path", pa.string()),
    ])
    lancedb_instance.create_table(TABLE_NAME, schema=schema)
    database = lancedb_instance[TABLE_NAME]

In [5]:
LEN_DATALOADER = len(dataloader)
SAVE_EVERY = int(0.05 * LEN_DATALOADER)

df = pd.DataFrame(columns=['embedding', 'video_name', 'image_name', 'frame_idx', 'path'])

for i, (images, video_names, image_names, frame_idxs, paths) in enumerate(tqdm(dataloader)):
    images = images.to(device)
    with torch.no_grad(), torch.amp.autocast('cuda'):
        embeddings = model.encode_image(images)
        embeddings /= embeddings.norm(dim=-1, keepdim=True)
        embeddings = embeddings.squeeze().cpu().numpy()
    data = {
        'embedding': [],
        'video_name': [],
        'image_name': [],
        'frame_idx': [],
        'path': []
    }
    for embedding, video_name, image_name, frame_idx, path in zip(embeddings, video_names, image_names, frame_idxs, paths):
        data['embedding'].append(embedding)
        data['video_name'].append(video_name)
        data['image_name'].append(image_name)
        data['frame_idx'].append(int(frame_idx))
        data['path'].append(path)
    df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
    if (i + 1) % SAVE_EVERY == 0 or i + 1 == LEN_DATALOADER:
        lancedb_instance[TABLE_NAME].add(df)
        df = pd.DataFrame(columns=['embedding', 'video_name', 'image_name', 'frame_idx', 'path'])
        print(f"Saved embeddings for batch {i+1}/{LEN_DATALOADER}")

  0%|          | 0/2879 [00:00<?, ?it/s]

c:\Users\minhh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


KeyboardInterrupt: 